In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.special as scsp

# Derivadas

In [3]:
def Derivative(f,x0,epsilon):
    """Calcula a derivada de f=f(x) em x0"""
    return (f(x0+epsilon)-f(x0-epsilon))/(2*epsilon)

def Jacobian(F,X0,e,m=0):
    """Determina a matriz Jacobiana da função 
    F = (F1,...,Fn)(x1,...,xm) em X0=(x01,...,x0m)"""
    if m == 0: m = X0.size
    E = np.identity(m)*e
    return (F(X0+E)-F(X0-E))/(2*e)

def SecondDerivative(f,x0,e,m=0,n=1):
    """Determina a segunda derivada de f=f(x) em x0,
    Ou as segundas derivadas parciais das funções 
    f=(f1,...,fn)(x1,...,xk) relativas a xm em x0"""
    if m == 0: m = x0.size
    E = np.identity(m)[n-1]*e
    print(x0)
    return (f(x0+E)+f(x0-E)-2*f(x0))/e**2

# Integrais

In [4]:
def TrapezoidalRule(a,b,N,f):
    """Integra ao longo do intervalo [a,b] dividido 
    em N-1 intervalos a função f usando a Regra do Trapézio
    Pode integrar para múltiplos valores de ou a ou b"""
    t, h = np.linspace(a,b,N,retstep=1)
    return np.sum(h*(f(t[:-1])+f(t[1:]))/2,axis=0)

def SimpsonRule(a,b,N,f):
    """Integra ao longo do intervalo [a,b] dividido 
    em 2N-1 intervalos a função f usando a Regra de Simpson
    Pode integrar para múltiplos valores de ou a ou b"""
    t, h = np.linspace(a,b,N, retstep=1)
    t0 = t[:-1]
    tf = t[1:]
    tm = (t0+tf)/2
    return np.sum((h*(f(t0)+4*f(tm)+f(tf)))/6, axis=0)

In [5]:
def TrapezoidalAdaptative(a,b,emin,f):
    """Integra ao longo do intervalo [a,b] a função f
    usando a Regra do Trapézio Adapatativa até que o 
    erro seja menor do que emin"""
    h = (b-a)
    I0 = ((f(a)+f(b))*h)/2
    n = 1
    #print("n\tI\te")
    #print(n,"\t",I0,"\t---")
    while True:

        N = 2*n+1
        t,h = np.linspace(a,b,N,retstep=1)
        
        I = I0 / 2 + np.sum(f(t[1:-1:2]))*h
        e = abs(I-I0)/3
        
        n *= 2
        #print(n,"\t",I,"\t",e)
        I0 = I
        if e<emin:
            return I
        
def SimpsonAdaptative(a,b,emin,f):
    """Integra ao longo do intervalo [a,b] a função f
    usando a Regra de Simpson Adapatativa até que o 
    erro seja menor do que emin"""
     
    S0 = (f(a)+f(b))/3
    T0 = 2*f((a+b)/2)/3
    I0 = (a-b)*(S0+2*T0)/2
    n = 2
    #print("n\tI\te")
    #print(n,"\t",I0,"\t---")
    while True:
        N = 2*n+1

        S = S0+T0
        S0 = S
        
        t, h = np.linspace(a,b,N,retstep=1)
        T = 2*np.sum(f(t[1:-1:2]))/3
        T0 = T

        I = h*(S+2*T)
        e = abs(I-I0)/15

        I0 = I
        n *= 2
        #print(n,"\t",I,"\t",e)
        if e<emin:
            return I
        
def Romberg(a,b,emin,f):
    """Integra ao longo do intervalo [a,b] a função f
    usando a Método de Romberg até que o erro seja menor do que emin"""
    h = (b-a)
    N = int(np.power(h/emin,1/4))
    R = np.zeros((N,N))
    R[0, 0] = ((f(a)+f(b))*h)/2
    n = 1
    i = 1
    #print("1  | ",R[0,0])
    while e > emin/10:
        Points = 2*n+1
        t,h = np.linspace(a,b,Points,retstep=1)
        R[i, 0] = R[i-1, 0] / 2 + np.sum(f(t[1:-1:2]))*h
        #print(i+1," | ",R[i,0],"\t",end="")
        for m in range(i):
            R[i, m+1] = R[i, m] + (R[i, m]-R[i-1, m])/(4**(m+1) - 1)
            #print(R[i,m+1],"\t",end="")
        #print()
        e = abs(R[i, i-1]-R[i-1, i-1])/(4**i-1)
        if e < emin:
            return R[i,i]
        i += 1
        n *= 2

In [6]:
def GaussianQuadrature(N,f,poly="Legendre",a=0,b=0):
    """Integra a funções f usando a Quadratura
    Gaussian utilizando N zeros do polinómio poly"""
    if poly=="Hermitenorm":
        x,w = scsp.roots_hermitenorm(N)         #Se houver necessidade definir uma nova função integranda
        return np.inner(f(x),w*np.exp(x**2/2))  #Que é a multiplicação da integranda por exp(x**2/2)
    if poly=="Hermite":
        x,w = scsp.roots_hermitenorm(N)         #Se houver necessidade definir uma nova função integranda
        return np.inner(f(x),w*np.exp(x**2))    #Que é a multiplicação da integranda por exp(x**2)
    if poly=="Legendre":
        x,w = scsp.roots_legendre(N)
    if poly=="Chebyshev_t":
        x,w = scsp.roots_chebyt(N)
        w *= np.sqrt(1-x**2)
    X = np.outer((b-a),x)/2 + np.outer((b+a),np.ones(x.shape))/2
    W = np.outer((b-a),w)/2
    I = np.squeeze(np.inner(f(X),W))
    
    if I.ndim != 1 and I.ndim!=0:
        return np.diag(I)
    else:
        return I

# Matrizes

In [44]:
def LUdecomposition(Mat):
    """Realiza a decomposição LU da matriz Mat
    e retorna a matriz de permutação P, a triangular superior U e a
    triangular inferior L tal que P@Mat=L@U"""
    
    N = Mat.shape[0]
    U = np.copy(Mat)
    L = np.zeros_like(Mat)
    P = np.identity(N,Mat.dtype)
    for i in range(N-1): 
        Max = np.argmax(abs(U[i:,i])) + i
        U[[i,Max],:] = U[[Max,i],:]
        P[[i,Max],:] = P[[Max,i],:]
        L[[i,Max],:] = L[[Max,i],:]
        mul = U[i:,i]/U[i,i]
        U[i+1:,:] -= np.outer(mul[1:],U[i,:])
        L[i:,i] = mul
    L[-1,-1] = 1
    return P,L,U

def QRdecomposition(A,method = "GramSchmidt"):
    """Realiza a decomposição QR da matriz A e retorna 
    a matriz ortogonal Q e a traingular superior R tal
    que A = QR"""
    N = A.shape[0]

    if method == "GramSchmidt":
        Q = np.zeros((N,N))
        R = np.zeros((N,N))
    
        for i in range(N):
            R[:i,i] = np.matmul(A[:,i],Q[:,:i])
            u = A[:,i] - np.sum(R[:i,i]*Q[:,:i],axis=1)    
            R[i,i] = np.linalg.norm(u)
            Q[:,i] = u/R[i,i]
    
    if method == "Householder":
        Q = np.identity(N)
        R = np.copy(A)
        
        for i in range(N):
            a = R[i:,i]
            u = np.zeros(N)
            
            ul = np.zeros(N-i)
            ul[0] = 1
            ul = a - ul*np.linalg.norm(a)
            ul = ul / np.linalg.norm(ul)
            
            u[i:] = ul
            H = np.identity(N)-2*np.outer(u,u)
            Q = np.matmul(Q,H)
            R = np.matmul(H,R)
   
    return Q,R

In [8]:
def Eigen(A,errmin):
    """Determina os valores próprios e respetivos
    vetores próprios da matriz A usando sucessivas
    decomposições QR com erro menor que errmin"""
    N = A.shape[0]
    Q = np.identity(N)
    M = np.copy(A)
    
    while True:
        Q = np.matmul(Q,QRdecomposition(M)[0])
        M = np.matmul(Q.T,np.matmul(A,Q))
        Diag = np.identity(N)*np.diagonal(M)
    
        if (np.abs(M-Diag)<errmin).all():
            return Q, M

In [248]:
def GaussElim(Mat,vec,method="partial"):
    """Resolve o sistema Mat*X=vec utilizando
    diferentes métodos"""
    Mat = np.copy(Mat)
    vec = np.copy(vec)
    if method == "total":
        n = vec.size
        M = np.zeros((n,n+1),Mat.dtype)
        M[:,:n] = np.copy(Mat)
        M[:,-1] = np.copy(vec)
        P = np.identity(n)
        for i in range(n):
            Max = np.argmax(abs(M[i:,i:n]))
            Maxpos = np.array((i+Max // (n-i),i+Max % (n-i)))
            M[[i,Maxpos[0]],:] = M[[Maxpos[0],i],:]
            M[:,[i,Maxpos[1]]] = M[:,[Maxpos[1],i]]  
            P[:,[i,Maxpos[1]]] = P[:,[Maxpos[1],i]]      
            M[i,:] /= M[i,i]
            M[i+1:,:] -= np.outer(M[i+1:,i],M[i,:])
            M[:i,:] -= np.outer(M[:i,i],M[i,:])
        return P@M[:,-1]
    if method == "multipletotal":
        T = vec.shape
        n = T[0]
        M = np.zeros((n,n+T[1]))
        M[:,:n] = np.copy(Mat)
        M[:,n:] = np.copy(vec)
        P = np.identity(n)
        for i in range(n):
            Max = np.argmax(abs(M[i:,i:n]))
            Maxpos = np.array((i+Max // (n-i),i+Max % (n-i)))
            M[[i,Maxpos[0]],:] = M[[Maxpos[0],i],:]
            M[:,[i,Maxpos[1]]] = M[:,[Maxpos[1],i]]  
            P[:,[i,Maxpos[1]]] = P[:,[Maxpos[1],i]]      
            M[i,:] /= M[i,i]
            M[i+1:,:] -= np.outer(M[i+1:,i],M[i,:])
            M[:i,:] -= np.outer(M[:i,i],M[i,:])
        return P@M[:,n:]
    N = vec.size 
    X = np.zeros(N,Mat.dtype)
    if method == "partial":
        for i in range(N-1): 
            if Mat[i,i] == 0:
                Max = np.argmax(abs(Mat[:,i]))
                Mat[[i,Max],:] = Mat[[Max,i],:]
                vec[[i,Max]] = vec[[Max,i]]

            mul = Mat[i+1:,i]
            # Para cada vetor linha v_j da matriz do sistema realiza a operação v_j = v_j - a_ji*v_i/a_ii, com j > i e a_ij o elemento da linha i e coluna j, ou seja, torna a matriz triangular
            Mat[i+1:,:],vec[i+1:] = Mat[i+1:,:] - np.outer(mul,Mat[i,:])/Mat[i,i], vec[i+1:] - mul*vec[i]/Mat[i,i]
            
            
        for i in range(N-1,-1,-1):
            # Determina o valor para cada variável (começando pelo fim)
            X[i] = (vec[i]-np.inner(X[i+1:],Mat[i,i+1:]))/Mat[i,i]
            
    if method == "LU":
        P,L,U = LUdecomposition(Mat)
        vec = P@vec
        X = np.zeros_like(vec)
        for i in range(0,N):
            X[i] = (vec[i]-np.inner(X[:i]*L[i,:i]))
        for i in range(N-1,-1,-1):
            X[i] = (X[i]-np.inner(X[i+1:]*U[i,i+1:]))/U[i,i]
    return X

def LUSolver(P,L,U,vec):
    N = vec.size
    vec = P@vec
    X = np.zeros_like(vec)
    for i in range(0,N):
        X[i] = (vec[i]-np.inner(X[:i],L[i,:i]))/L[i,i]
    for i in range(N-1,-1,-1):
        X[i] = (X[i]-np.inner(X[i+1:],U[i,i+1:]))/U[i,i]
    return X


# Zeros e Extremos de funções

In [10]:
def relaxation(F,X,errmin,retit = 0,stop=1e3):
    """Determina um zero da equação F(X)=X pelo método
    da relaxação"""
    n = 0
    while True:
        X, err = F(X), np.sqrt(np.sum((F(X)-X)**2,axis=0))
        n += 1
        if n == stop: return np.nan
        if (err < errmin).all():
            if retit:
                return X,n
            return X
        
def super_relaxation(f,errmin,x=1,omega = 1,retit = 0):
    """Determina um zero da equação f(x)=x pelo método
    da sobre-relaxação"""
    if omega == -1:
        if retit:
            return x,np.inf
        return x
    n = 0
    while True:
        x, err = (1+omega)*f(x)-omega*x, abs(f(x)-x)
        n += 1
        if (err < errmin).all():
            if retit:
                return x,n
            return x


In [11]:
def Bissection(f,a,b,errmin):
    """Determina um zero de f pelo método
    da bisseção começando a partir de a e b"""
    if f(a) > f(b):
        a, b = b, a
    while True:
        h = (a+b)/2
        if f(h) == 0: return h
        elif f(h) < 0:
            a = h
        else:
            b = (a+b)/2
        if abs(f(h)) < errmin:
            return h

def SecantMethod(f,xn,xn1,errmin):
    """Determina um zero de f pelo método da
    secante começando a partir de xn e xn1"""
    while True:
        xn1, xn = xn1 - f(xn1) * (xn1-xn)/(f(xn1)-f(xn)), xn1
        if abs(xn1-xn) < errmin:
            return xn1

In [12]:
def NewtonMethod(f,x,errmin):
    """Determina um zero de f pelo método de 
    Newton começando a partir de x"""
    while True:
        df = (f(x+errmin)-f(x-errmin))/(2*errmin)
        k = x
        x = x - f(x)/df
        if abs(x-k) < errmin:
            return x

def NDNewtonMethod(F,X,errmin, m = 0):
    """Determina um zero de F pelo método de
    Newton N-dimensional a partir de X"""
    if m == 0: m = X.size
    J = Jacobian(F,X,m)
    while True:
        X, k = X - np.linalg.solve(J,F(X)), X
        if (abs(X-k)<errmin).all():
            return X

In [13]:
def GoldenSection(f,x1,x4,errmin):
    """Determina um mínimo entre [x1,x4] de f
    pelo método da Secção Aúrea"""
    delta = (3-np.sqrt(5))/2
    x2, x3 = x1+delta*(x4-x1),x4-delta*(x4-x1)
    while x4-x1>errmin:
        if f(x2) > f(x3):
            x4 = x3
            x3 = x2
            x2 = x1+delta*(x4-x1)
        else:
            x1 = x2
            x2 = x3
            x3 = x4-delta*(x4-x1)
    return 0.5*(x1+x4)

# Transformadas de Fourier

In [14]:
def FourierTransform(f,a,b,N,k):
    """Determina os coeficientes da Transformada
    de Fourier da função f no intervalo [a,b]"""
    L = b-a
    g = np.arange(-k,k+1)
    
    g = SimpsonRule(a,b,N,lambda x : f(x) * np.exp(-2*np.pi*np.outer(g,x)*1j/L)) / L

    return g

def FourierTransformArray(v,k):
    """Determina os coeficientes da Transformada
    de Fourier da função f no intervalo [a,b],
    quando f é definida por um array"""
    L = v.size
    g = np.arange(-k,k+1)
    t = np.arange(0,L)
    f = lambda n : v[n]*np.exp(-2*np.pi*np.outer(g,n)*1j/L)
    g = np.sum(f(t[:-1:2])+f(t[2::2])+4*f(t[1::2]),axis=1)/(6*L)
    return g

def Smoothing(f,r):
    """Suaviza a função f utilizando r*100 % 
    dos termos da sua Transformada de Fourier"""
    k = np.fft.rfft(f)
    k[int(k.size*r)+1:] *= 0 
    nk = np.fft.irfft(k)
    return nk

# Equações Diferenciais

In [15]:
def EulerMethod(f,t0,x0,tf,h):
    """Determina a solução da equação diferencial ordinária
    dx/dt = f(x,t) no intervalo [t0,tf] com passo h e condições
    iniciais (t0,x0) utilizando o método de Euler"""    
    T = np.arange(t0,tf,h)
    X = np.zeros((T.size,np.array(x0).size))
    X[0,:] = x0
    for i in range(len(T)-1):
        X[i+1]=X[i]+h*f(X[i])
    return T,X

def RungeKutta4(f,t0,x0,tf,h):
    """Determina a solução da equação diferencial ordinária
    dx/dt = f(x,t) no intervalo [t0,tf] com passo h e condições
    iniciais (t0,x0) utilizando o método de Runge-Kutta de 4ª ordem"""
    T = np.arange(t0,tf,h)
    X = np.zeros((T.size,np.array(x0).size))
    X[0,:] = x0
    for i in range(len(T)-1):
        k1 = f(X[i]+h,T[i+1])
        k2 = f(X[i]+h*k1/2,T[i]+h/2)
        k3 = f(X[i]+h*k2/2,T[i]+h/2)
        k4 = f(X[i]+h*k3,T[i+1])
        X[i+1] = X[i] + (k1+2*k2+2*k3+k4)*h/6
    return T,X

def ShootingMethod(f,t0,tf,h,x0,xf,ymin,ymax,errmin):
    def get_xf(f,t0,x0,y0,tf,h):
        T = np.arange(t0,tf,h)
        X = np.array((x0,y0))
        for i in range(len(T)-1):
            k1 = f(X+h,T[i+1])
            k2 = f(X+h*k1/2,T[i]+h/2)
            k3 = f(X+h*k2/2,T[i]+h/2)
            k4 = f(X+h*k3,T[i+1])
            X = X + (k1+2*k2+2*k3+k4)*h/6
        return X[0]
    
    xmin, xmax = get_xf(f,t0,x0,ymin,tf,h), get_xf(f,t0,x0,ymax,tf,h)
    while abs(xmin-xmax)>errmin:
        yp = (ymin+ymax)/2
        xp = get_xf(f,t0,x0,yp,tf,h)
        
        if (xp-xf)*(xmin-xf)>0:
            xmin = xp
            ymin = yp
        else:
            xmax = xp
            ymax = yp            
    return (ymin+ymax)/2
"""
def RungeKutta4Adaptative(f,t0,x0,tf,hdelta):
    h = (tf-t0)*1e-4
    N = int(hdelta/h)
    T = np.zeros(N)
    T[0] = t0
    X = np.zeros((N,np.array(x0).size))
    X[0] = x0
    i = 0
    while T[i]<tf:
        while True:
            k1 = f(X[i]+h,T[i]+h)
            k2 = f(X[i]+h*k1/2,T[i]+h/2)
            k3 = f(X[i]+h*k2/2,T[i]+h/2)
            k4 = f(X[i]+h*k3,T[i]+h)
            x1 = X[i] + (k1+2*k2+2*k3+k4)*h/6
            k1 = f(x1+h,T[i]+h)
            k2 = f(x1+h*k1/2,T[i]+h/2)
            k3 = f(x1+h*k2/2,T[i]+h/2)
            k4 = f(x1+h*k3,T[i]+h)
            x1 += (k1+2*k2+2*k3+k4)*h/6
            k1 = f(X[i]+2*h,T[i]+2*h)
            k2 = f(X[i]+h*k1,T[i]+h)
            k3 = f(X[i]+h*k2,T[i]+h)
            k4 = f(X[i]+2*h*k3,T[i]+2*h)
            x2 = X[i] + (k1+2*k2+2*k3+k4)*h/3
            rho = 30*hdelta/np.linalg.norm(x2-x1)
            if rho > 1:
                T[i+1]=T[i] + 2*h
                X[i+1]=x1
                h*=rho
                i+=1
                break
            h *= rho
    return T,X
"""